In [2]:
conda install -c microsoft azure-cosmos

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [8]:
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import datetime
import time; time.time()

import config

# ----------------------------------------------------------------------------------------------------------
# Prerequistes -
#
# 1. An Azure Cosmos account -
#    https://docs.microsoft.com/azure/cosmos-db/create-cosmosdb-resources-portal#create-an-azure-cosmos-db-account
#
# 2. Microsoft Azure Cosmos PyPi package -
#    https://pypi.python.org/pypi/azure-cosmos/
# ----------------------------------------------------------------------------------------------------------
# Sample - demonstrates the basic CRUD operations on a Item resource for Azure Cosmos
# ----------------------------------------------------------------------------------------------------------


HOST = config.settings['host']
MASTER_KEY = config.settings['master_key']
DATABASE_ID = config.settings['database_id']
CONTAINER_ID = config.settings['container_id']





In [9]:
print(CONTAINER_ID)

Protest


In [10]:
# Define a SQL query that counts all documents
query = "SELECT VALUE COUNT(1) FROM c "

# Execute the query and get the result
result = container.query_items(query=query, enable_cross_partition_query=True)
count = list(result)[0]

# Print the count
print(f"There are {count} documents in the collection.")

NameError: name 'container' is not defined

# Valores de referencia para testear TTL

### select count(1) from c where c._ts <> 1674043294
### select count(1) from c where c._ts >= 1674045850  

- Total documentos 419507   -- = 292  >   419215   < 0 

- 1674043294 --> 292
- 1674043295 --> 206 
- 1674043296 --> 217
- 1674043297 --> 142
- 1674043299 --> 355
- 1674043300 --> 77
- 1674043301 --> 222
- _ts >= 1674045850 --> 10518 


### To calculate when a document will be deleted by TTL based on its _ts value, you need to know two things: 
-  _ts value itself and the 
- TTL value for the container or the item. 

. The _ts value is an epoch value in seconds since an item was last modified1. 

. The TTL value is the number of seconds after which an item expires2. 

To find out when a document will be deleted, you need to add the _ts value and the TTL value together and convert them to a human-readable date format. For example, if your _ts value is 1674043294 seconds and your TTL value is 86400 seconds (one day), then your document will be deleted on March 9th, 2023 at 11:01:34 GMT. You can use online tools like https://www.epochconverter.com/ to convert epoch values to dates easily.

## Intento leer 30 documentos , y actualizar el id en base maximo id recuperado de la bbdd

###  select max(c.id) from c 
###  item['id'] += 1 
### 

In [3]:
# Define a SQL query that counts all documents
query = "SELECT VALUE COUNT(1) FROM c "

# Execute the query and get the result
result = container.query_items(query=query, enable_cross_partition_query=True)
count = list(result)[0]

# Print the count
print(f"There are {count} documents in the collection.")

NameError: name 'container' is not defined

In [2]:
# Define a SQL query that counts all documents
query = 'select max(c.id) as maxid from c'   


# Execute the query and get the result
result = container.query_items(query=query, enable_cross_partition_query=True)


#maxidcalculated = item['c.id']
# Print the result
#print(f"There are {item['_ts']} documents in the collection with _ts =1674043294")
#print(item['c.id'])


#query = 'select top 10 c.url,c.date,c.content, c.renderedcontent, c.id, c.user ,c._rid_ ,c._self, \
#c._etag_, c._attachments,c._ts from c'  

#result = container.query_items(query=query, enable_cross_partition_query=True)

#for item in result:
#    print(result_list)
#  current_date = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(item['_ts']))
  #print(( f" Doc id y epoch caducidad: {item['repes'], item['_ts']}"))
#  print( f" Repeticiones:  {item['repes']} y fecha caducidad:{current_date}")
  #print((item['repes'], item['_ts']))

#result_list = list(result)
print(result_list)
print(item['id'])

NameError: name 'container' is not defined

In [11]:

def query_items(container, account_number):
    print('\nQuerying for an  Item by Partition Key\n')

    # Including the partition key value of account_number in the WHERE filter results in a more efficient query
    items = list(container.query_items(
        query="SELECT * FROM r WHERE r.partitionKey=@account_number",
        parameters=[
            { "name":"@account_number", "value": account_number }
        ]
    ))

    print('Item queried by Partition Key {0}'.format(items[0].get("id")))


def replace_item(container, doc_id, account_number):
    print('\nReplace an Item\n')

    read_item = container.read_item(item=doc_id, partition_key=account_number)
    read_item['subtotal'] = read_item['subtotal'] + 1
    response = container.replace_item(item=read_item, body=read_item)

    print('Replaced Item\'s Id is {0}, new subtotal={1}'.format(response['id'], response['subtotal']))


def upsert_item(container, doc_id, account_number):
    print('\nUpserting an item\n')

    read_item = container.read_item(item=doc_id, partition_key=account_number)
    read_item['subtotal'] = read_item['subtotal'] + 1
    response = container.upsert_item(body=read_item)

    print('Upserted Item\'s Id is {0}, new subtotal={1}'.format(response['id'], response['subtotal']))


#def delete_item(container, doc_id, account_number):
 #   print('\nDeleting Item by Id\n')

  #  response = container.delete_item(item=doc_id, partition_key=account_number)

   # print('Deleted item\'s Id is {0}'.format(doc_id))


def get_sales_order(item_id):
    order1 = {'id' : item_id,
            'partitionKey' : 'Account1',
            'purchase_order_number' : 'PO18009186470',
            'order_date' : datetime.date(2005,1,10).strftime('%c'),
            'subtotal' : 419.4589,
            'tax_amount' : 12.5838,
            'freight' : 472.3108,
            'total_due' : 985.018,
            'items' : [
                {'order_qty' : 1,
                    'product_id' : 100,
                    'unit_price' : 418.4589,
                    'line_price' : 418.4589
                }
                ],
            'ttl' : 60 * 60 * 24 * 30
            }

    return order1


def get_sales_order_v2(item_id):
    # notice new fields have been added to the sales order
    order2 = {'id' : item_id,
            'partitionKey' : 'Account2',
            'purchase_order_number' : 'PO15428132599',
            'order_date' : datetime.date(2005,7,11).strftime('%c'),
            'due_date' : datetime.date(2005,7,21).strftime('%c'),
            'shipped_date' : datetime.date(2005,7,15).strftime('%c'),
            'subtotal' : 6107.0820,
            'tax_amount' : 586.1203,
            'freight' : 183.1626,
            'discount_amt' : 1982.872,
            'total_due' : 4893.3929,
            'items' : [
                {'order_qty' : 3,
                    'product_code' : 'A-123',      # notice how in item details we no longer reference a ProductId
                    'product_name' : 'Product 1',  # instead we have decided to denormalise our schema and include
                    'currency_symbol' : '$',       # the Product details relevant to the Order on to the Order directly
                    'currency_code' : 'USD',       # this is a typical refactor that happens in the course of an application
                    'unit_price' : 17.1,           # that would have previously required schema changes and data migrations etc.
                    'line_price' : 5.7
                }
                ],
            'ttl' : 60 * 60 * 24 * 30
            }

    return order2

def run_sample():
    client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY}, user_agent="CosmosDBPythonQuickstart", user_agent_overwrite=True)
    try:
        # setup database for this sample
        try:
            db = client.create_database(id=DATABASE_ID)
            print('Database with id \'{0}\' created'.format(DATABASE_ID))

        except exceptions.CosmosResourceExistsError:
            db = client.get_database_client(DATABASE_ID)
            print('Database with id \'{0}\' was found'.format(DATABASE_ID))

        # setup container for this sample
        try:
            container = db.create_container(id=CONTAINER_ID, partition_key=PartitionKey(path='/partitionKey'))
            print('Container with id \'{0}\' created'.format(CONTAINER_ID))

        except exceptions.CosmosResourceExistsError:
            container = db.get_container_client(CONTAINER_ID)
            print('Container with id \'{0}\' was found'.format(CONTAINER_ID))

        scale_container(container)
        create_items(container)
        read_item(container, 'SalesOrder1', 'Account1')
        read_items(container)
        query_items(container, 'Account1')
        replace_item(container, 'SalesOrder1', 'Account1')
        upsert_item(container, 'SalesOrder1', 'Account1')
        delete_item(container, 'SalesOrder1', 'Account1')

        # cleanup database after sample
        try:
            client.delete_database(db)

        except exceptions.CosmosResourceNotFoundError:
            pass

    except exceptions.CosmosHttpResponseError as e:
        print('\nrun_sample has caught an error. {0}'.format(e.message))

    finally:
            print("\nrun_sample done")


if __name__ == '__main__':
    run_sample()


Database with id 'ToDoList' was found
Container with id 'Items' was found

run_sample done


NameError: name 'scale_container' is not defined